In [1]:
import docx
import os
from openpyxl import load_workbook

In [2]:
doc_path = 'teste_8.docx'

In [3]:
class PlanilhaNaoEncontrada(ValueError):
    '''Raised quando nao encontra a planilha para a meta'''


class ParserTables:
    
    table_mapper  = {
        0 : 'Meta ',
        1 : 'Objetivo estratégico',
        2 : 'Meta',
        3 : 'Indicador',
        4 : 'Contexto',
        5 : 'Informações Complementares',
        6 : 'ODS vinculados',
        7 : 'Iniciativas',
        8 : 'Secretaria Responsável',
        9 : 'Total Orçado',
        10 : 'Regionalização - projeção quadriênio'
    }
    
    def __init__(self, doc_path):
        
        self.doc = docx.Document(doc_path)
        
    def parse_tables(self, result_list = None):
        
        if result_list is None:
            result_list = list()
        
        for table in self.doc.tables:
            parsed = {}
            for i, row in enumerate(table.rows):
                cells = row.cells
                row_label = cells[0].text
                row_value = cells[1].text
                
                if row_label != self.table_mapper[i]:
                    raise ValueError(f'Tabela fora do formato : ({row_label}  {row_value})')
                
                parsed[row_label] = row_value
                
            result_list.append(parsed)
        
        return result_list
    
    def parse_iniciativas(self, txt_iniciativas):
    
        parsed = {}
        ini_list = txt_iniciativas.split('\n')

        for ini in ini_list:
            try:
                letra, val = ini.split(') ')[:2]
                parsed[letra] = val
            except ValueError:
                print(ini)
                print(ini_list)

        return parsed
    
    def parse_region(self, txt_regionalizacao):
        
        parsed = {}
        
        reg_list = txt_regionalizacao.split('\n')
        for reg in reg_list:
            
            subs, val = reg.split(' : ')
            parsed[subs] = val
            
        
        return parsed
    
    def __call__(self):
                
        parsed = self.parse_tables()
        
        key_reg = self.table_mapper[10]
        key_ini = self.table_mapper[7]
        
        for table in parsed:
            table[key_reg] = self.parse_region(table[key_reg])
            table[key_ini] = self.parse_iniciativas(table[key_ini])
            
        return parsed
    
class ExcelInputer:
    
    
    xl_mapper = {
        'Meta ' : 'C5',
        #'Objetivo estratégico' : None,
        'Meta' : 'C6',
        'Indicador' : 'C9',
        'Contexto' : 'C10',
        'Informações Complementares' : 'C11',
        #'ODS vinculados' : None,
        #'Iniciativas' : 'solve_iniciativas',
        'Secretaria Responsável' : 'C4',
        #'Total Orçado' : None,
        #'Regionalização - projeção quadriênio' : 'solve_regionalizacao'
        
    }
    
    def __init__(self, parsed_data, folder_original, folder_salvar):
        
        
        self.planilhas = self.get_planilhas(folder_original)
        self.folder_salvar = self.solve_folder(folder_salvar)
        self.data = parsed_data
        
    def solve_folder(self, folder):
        
        if not os.path.exists(folder):
            os.mkdir(folder)
            
        return folder
        
    def get_planilhas(self, folder_original):
        
        fpaths =  [os.path.join(folder_original, file) for file in 
                    os.listdir(folder_original) if 'xls' 
                    in file.split('.')[-1]]
        
        return fpaths
    
    def read_workbook(self, file):
        
        wb = load_workbook(file, read_only=False, data_only=True)
        
        return wb
    
    def match_planilha(self, wb, num_meta):
        
        sheet = wb['Ficha técnica Meta ']
        
        num_meta_planilha = sheet['C5'].value
        
        if str(num_meta_planilha) == num_meta:
            return True
        
    def find_planilha(self, num_meta):
        
        for file in self.planilhas:
            wb = self.read_workbook(file)
            
            if self.match_planilha(wb, num_meta):
                return file, wb
        else:
            raise PlanilhaNaoEncontrada(f'Planilha para a meta {num_meta} não encontrada')
            
    def save_wb(self, wb, orig_file_path):
        
        orig_f_name = os.path.split(orig_file_path)[-1]
        save_path = os.path.join(self.folder_salvar, orig_f_name)
        
        wb.save(save_path)
        
        wb.close()
        
        
            
    def insert_data(self, sheet, parsed_data):
        
        
        for nome_campo in parsed_data:
            if nome_campo in self.xl_mapper:
                celula = self.xl_mapper[nome_campo]
                valor = parsed_data[nome_campo]
                sheet[celula] = valor
                
    def __call__(self):
        
        for parsed in self.data:
            
            num_meta = parsed['Meta ']
            print(f'escrevendo {num_meta}')
            try:
                file, wb = self.find_planilha(num_meta)
            except PlanilhaNaoEncontrada as e:
                print(e)
                continue
            sheet = wb['Ficha técnica Meta ']
            self.insert_data(sheet, parsed)
            self.save_wb(wb, file)
            
                

In [4]:
p = ParserTables(doc_path)

In [5]:
parsed_data = p()

Programa WiFi Livre SP;
['a) Realizar diagnóstico e elaborar plano de revisão e expansão do', 'Programa WiFi Livre SP;', 'b) Unificar e definir padrões mínimos para as iniciativas de Wi-Fi Público das Secretarias;', 'c) Sinalizar com placas indicativas todas as localidades do Programa Wi-Fi Livre SP;', 'd) Estruturar e realizar ações de divulgação para expandir o número de usuários do Programa Wi-fi Livre SP.']
ao cidadão;
['a) Adequar o padrão de atendimento a fim de garantir excelência no atendimento', 'ao cidadão;', 'b) Atualizar e modernizar o parque tecnológico;', 'c) Padronizar a comunicação visual para torná-la mais simples e acessível;', 'd) Realizar adequação básica do layout físico das praças para otimização dos atendimentos.']


In [6]:
e = ExcelInputer(parsed_data,
                'original_data/Devolutiva 11-jun/',
                'teste_retorno_2')

In [ ]:
e()

escrevendo 1


In [5]:
path_one_drive = r'C:\Users\h-pgy\one_drive_prefs\OneDrive - Default Directory\Shared Documents\Estruturação do PDM 2021-2024\Elaboração PDM Versão Final\Fichas Metas'

In [6]:
import os

In [7]:
os.listdir(path_one_drive)

['Devolutiva 11-jun']